In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
master_phe_info_f <- '@@@@@@@@@@@@@@@@@@@'


In [5]:
master_phe_info_df <- fread(master_phe_info_f) %>%
rename('GBE_ID' = '#GBE_ID')

In [7]:
# This file is a copy from 
# UK Biobank Category 1307
# Infectious Disease Antigens - Infectious Diseases - Blood assays - Assay results - Biological samples
# https://biobank.ctsu.ox.ac.uk/crystal/label.cgi?id=1307

list_of_fields <- fread('UKB_Category_1307.tsv') %>%
rename('Field_ID' = '#Field_ID')


In [10]:
library(googlesheets)
gs_auth(token = "@@@@@@@@@@@@@@@@@@@")

file <- '@@@@@@@@@@@@@@@@@@@'
GBE_names_df <- file %>% gs_url() %>% gs_read(ws = 'GBE_names')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: @@@@@@@@@@@@@@@@@@@
Sheet successfully identified: "GBE_names"
Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_name_len = col_double()
)


In [23]:
PheWAS_traits <- master_phe_info_df %>% 
filter(FIELD %in% list_of_fields$Field_ID) %>%
left_join(
    GBE_names_df %>%
    rename('phenotype' = 'GBE_short_name', 'phenotype_full' = 'GBE_NAME') %>%
    select(GBE_ID, phenotype, phenotype_full),
    by='GBE_ID'
) %>%
rename('N_WB' = 'N_GBE', 'UKB_Field' = 'FIELD') %>%
select(GBE_ID, phenotype, phenotype_full, UKB_Field, N, N_WB, N_AFR, N_SAS, N_EAS)


In [25]:
setdiff(list_of_fields$Field_ID, PheWAS_traits$UKB_Field)

[1] 23049 23048

In [27]:
PheWAS_traits %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite('PRS_PheWAS.phenotypes.tsv', sep='\t')